In [27]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [28]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

#constants
c = 299792.458

In [29]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [30]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [112]:
tf_mastertable = vstack([tf_fuji,tf_iron])

In [113]:
tf_mastertable['VELOCITY']=np.nan
tf_mastertable['VELOCITY_REL']=np.nan

In [114]:
# set up dictionary of SGA ID
'''tf_fuji_dict = {}
for sga_id in np.unique(tf_fuji['SGA_ID']):
    tf_fuji_dict[sga_id] = np.where(tf_fuji['SGA_ID'] == sga_id)[0]

tf_iron_dict = {}
for sga_id in np.unique(tf_iron['SGA_ID']):
    tf_iron_dict[sga_id] = np.where(tf_iron['SGA_ID'] == sga_id)[0]
'''
tf_mastertable_dict = {}
for sga_id in np.unique(tf_mastertable['SGA_ID']):
    tf_mastertable_dict[sga_id] = np.where(tf_mastertable['SGA_ID'] == sga_id)[0]

In [108]:
# apply mask
'''
filtered_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_fuji = filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE'] == 1]
filtered_tf_iron = filtered_tf_iron[filtered_tf_iron['ROT_CURVE'] == 1]
#print(len(filtered_tf_fuji))
'''

1330


In [120]:
# pull out all unique SGA ID's for iron and fuji
#tf_fuji_rot = np.unique(filtered_tf_fuji['SGA_ID'])
#tf_iron_rot = np.unique(filtered_tf_iron['SGA_ID'])
# combine iron and fuji unique IDs (don't need if processing seperately?)
'''tf_galaxies = np.unique(np.concatenate((tf_fuji_rot, tf_iron_rot)))
'''
tf_galaxies = np.unique(tf_mastertable['SGA_ID'])

#define targetid
#targetid_fuji = targetid in tf_fuji_rot
#targetid_iron = targetid in tf_iron_rot

#identify targets
'''
for sga_id in tf_fuji_dict:
    targets_fuji = tf_fuji[tf_fuji_dict[sga_id]]
for sga_id in tf_iron_dict:
    targets_iron = tf_iron[tf_iron_dict[sga_id]]
'''
# loop through each SGA ID
#for sga_id in tf_fuji_dict:
for sga_id in tf_galaxies:
                            
    #identify z values of targets
    '''
        z_targets_fuji=tf_fuji['Z'][tf_fuji_dict[sga_id]]
    z_targets_iron=tf_iron['Z'][tf_iron_dict[sga_id]]
    '''
    z_targets=tf_mastertable['Z'][tf_mastertable_dict[sga_id]]
    #identify good centers
    '''
    fuji_centers_b = tf_fuji['SKY_FIBER_DIST_R26'][tf_fuji_dict[sga_id]] < 0.001
    fuji_good_b = (tf_fuji['ZWARN'][tf_fuji_dict[sga_id]]==0) & (tf_fuji['DELTACHI2'][tf_fuji_dict[sga_id]]>25)
    fuji_good_centers = tf_fuji[tf_fuji_dict[sga_id]][fuji_centers_b & fuji_good_b] 
    
    iron_centers_b = tf_iron['SKY_FIBER_DIST_R26'][tf_iron_dict[sga_id]] < 0.001
    iron_good_b = (tf_iron['ZWARN'][tf_iron_dict[sga_id]]==0) & (tf_iron['DELTACHI2'][tf_iron_dict[sga_id]]>25)
    iron_good_centers = tf_iron[tf_iron_dict[sga_id]][iron_centers_b & iron_good_b] 
    '''
    centers_b = tf_mastertable['SKY_FIBER_DIST_R26'][tf_mastertable_dict[sga_id]] < 0.001
    good_b = (tf_mastertable['ZWARN'][tf_mastertable_dict[sga_id]]==0) & (tf_mastertable['DELTACHI2'][tf_mastertable_dict[sga_id]]>25)
    good_centers = tf_mastertable[tf_mastertable_dict[sga_id]][centers_b & good_b] 
    
    #defining terms
    '''
    if len(fuji_good_centers) > 0:
        z_fuji = fuji_good_centers['Z']
        z_fuji_err = fuji_good_centers['ZERR']
        w_fuji = np.sum(1 / (z_fuji_err**2))
    else:
        z_fuji = np.nan * np.ones(1)
        z_fuji_err = np.nap * np.ones(1)
        w_fuji = 0
    if len(iron_good_centers) > 0:
        z_iron = iron_good_centers['Z']
        z_iron_err = iron_good_centers['ZERR']
        w_iron = np.sum(1 / (z_iron_err**2))
    else:
        z_iron = np.nan * np.ones(1)
        z_iron_err = np.nap * np.ones(1)
        w_iron = 0
    '''
    z = good_centers['Z']
    z_err = good_centers['ZERR']
    # calculating the weights
    #w = np.sum(w_fuji + w_iron)
    w = np.sum(1 / (z_err**2))
    
    # calculating the weighted average z naught
    z_o = (np.nanmean(w * z)) / w
    if sga_id == 123:
        print(z_o)
    #find delta z from z naught and z values of all viable targets 
    '''
    delta_z_fuji = z_targets_fuji - z_o
    delta_z_iron = z_targets_iron - z_o
    '''
    delta_z = z_targets - z_o
    
    #use delta z to find velocity and put into column
    '''
    velocity_fuji = c * delta_z_fuji
    velocity_iron = c * delta_z_iron
    '''
    velocity = c * delta_z
    # update the original tables with velocities
    '''
    tf_fuji['VELOCITY'][tf_fuji_dict[sga_id]]=velocity_fuji
    tf_iron['VELOCITY'][tf_iron_dict[sga_id]]=velocity_iron
    '''
    tf_mastertable['VELOCITY'][tf_mastertable_dict[sga_id]]=velocity

/tmp/ipykernel_2267510/1001609839.py:70: RuntimeWarning: Mean of empty slice
  z_o = (np.nanmean(w * z)) / w


0.1099660983385235


In [121]:
tf_mastertable[tf_mastertable_dict[123]]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1071233026228227,242.564878755415,53.6763270782975,10153,sv3,0.184971788085982,4.36696398757541e-05,4,0.458266502246261,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56487875541504,53.67632707829753,1.0,0.0051345584203947876,1.0000000466304622,1.0,0.0,22486.14009337598,nan
1071233026228228,242.564908359756,53.6746327647682,10153,sv3,1.5161235823035,9.98100168252869e-05,4,0.224537968635559,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.5649083597559,53.67463276476823,1.0,0.0034401541416619126,0.6700000312419042,1.0,0.0,421555.40845295583,nan
1071233026228231,242.565028546859,53.6677528249076,10153,sv3,0.223582059648363,6.6695997421467e-05,4,5.54895717371255,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56502854685928,53.667752824907645,1.0,0.0034401541416594996,0.6700000312414343,1.0,0.0,34061.20830910968,nan
1084427165761544,242.565058139153,53.6660585113414,10153,sv3,0.125989095083328,3.5227684897735e-05,4,5.68882738612592,fuji/healpix/sv3/dark/101/10153/redrock-sv3-dark-10153.fits,TFT,123,242.56505813915317,53.66605851134143,1.0,0.0051345584203787275,1.0000000466273342,1.0,0.0,4803.573578650942,nan
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,-0.19004118687180754,nan
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,0.190041186875968,nan


In [11]:
#define z naught error
z_o_err = np.sum(np.sqrt(1 / (w_fuji)) + np.sqrt(1 / (w_iron)))

#define z error for all targets
z_err_fuji = z_targets_fuji['ZERR']
z_err_iron = z_targets_iron['ZERR'] 

#define z error combined

#find v error for each target
v_error_1 = ((z_o_err**2) + (z_err**2))
v_err=c*(v_error_1**0.5)
print(v_err)
# put v error into new column for each table

       ZERR       
------------------
 15.18073317579169
 30.89339957073999
 21.42094739439493
13.061136285094975
  8.61991366467759
  8.56454648955498
